## Data Processing --- Calculate Beta from 3 factor model in bond market 

#### Include factors from Fama and French(1993):  
Term:  
    diff between monthly ***long-term government bond*** return and the one-month ***treasury bill rate*** measured at the end of the previous month 
    
DEF:  
    diff between a ***market portfolio of long-term corporate bonds*** and the ***long-term government bond*** return 
    
Excess bond market return：   
    value weighted ***average return of all corporate bonds*** in excess of the ***one month T-bill***
    
Components are constructed as follows:   
    ***long-term government bond*** : return of '银行间国债指数_7-10' from resset 
    ***treasury bill rate***: 1. 1/12 of the '国开债（1月）到期收益率' from wind 2. adopt the risk_free rate in the dataset
    ***market portfolio of long-term corporate bonds***: return of '总指数_7-10年' from resset 
    ***average return of all corporate bonds*** : two ways: 1. return of '信用债指数' from resset 2. value-weighted return from our dataset 
    
Since we have two ways to calculate the average return of all corporate bonds, I ran two regressions. The first one incorprate the excess bond market return calculated by 信用债指数, the second way is calculated by the value-weighted return.    
We also have two measures on the treasury bill rate. So we run another two types of regressions. 

#### The beta result is explained as follows: 
    1. beta1(_name of factor)
    rt - rf1 = beta1_term(long-term gov - rf1) + beta1_def(long term corp - long term gov) + beta1_excess(corpor1 - rf1)  
    rf1: 国开债（1月）到期收益率
    corpr1: return of '信用债指数' from resset
    
    2. beta2(_name of factor)
    rt - rf2 = beta2_term(long-term gov - rf2) + beta2_def(long term corp - long term gov) + beta2_excess(corpor1 - rf2)  
    rf2: risk free rate in dataset 
    corpr1: return of '信用债指数' from resset
    
    3. beta3(_name of factor)
    rt - rf1 = beta3_term(long-term gov - rf1) + beta3_def(long term corp - long term gov) + beta3_excess(corpor2 - rf1)  
    rf1: 国开债（1月）到期收益率
    corpr2: value-weighted return from dataset
    
    4. beta4(_name of factor)
    rt - rf2 = beta4_term(long-term gov - rf2) + beta4_def(long term corp - long term gov) + beta4_excess(corpor2 - rf2)  
    rf2: risk free rate in dataset 
    corpr2: value-weighted return from dataset
    
    


We haven't included the the MOM factor and Illiq factors. According to the previous study, MOM factor should be calculated based on the  5*5 bivaraite sort portfolios of credit ratings and momentum. But our data cannot be sorted by credit rating(most ratings are over AA+ ). Illiquidity beta need a 5 year rolling window. 
We can add these two factor in later adjustment. 
    

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

## Import Index Data

In [2]:
data_address = './data/index/'
data_list = glob.glob(data_address + r"index*.csv")
raw_index = pd.DataFrame([])
for file in data_list:
    print(file)
    raw_index = raw_index.append(pd.read_csv(file))
    
data_address = '~/Downloads/data/'
tbill = pd.read_excel(data_address + 'treasury_bill.xlsx')

./data/index/index1.csv
./data/index/index2.csv
./data/index/index3.csv
./data/index/index7.csv
./data/index/index6.csv
./data/index/index4.csv
./data/index/index5.csv
./data/index/index8.csv
./data/index/index9.csv


In [3]:
special_notations = ['_', '(', ')', '%', '/']
for i in special_notations:
    raw_index.columns=raw_index.columns.str.replace(i,'')
    tbill.columns=tbill.columns.str.replace(i,'')
tbill = tbill.iloc[1:-2]
display(raw_index.columns)
display(tbill.columns)
tbill.columns = ['time', 'tb1month', 'tb10yrs', 'tb5yrs', 'cbaap5yrs', 'cbap5yrs']
tbill['time'] = tbill['time'].apply(lambda x: pd.to_datetime(x))
tbill['year'] =tbill['time'].apply(lambda x: x.year)
tbill['month'] = tbill['time'].apply(lambda x: x.month)

display(tbill.columns)

#中债中短期票据到期收益率A+:5年 ： market portfolio of long-term corporate bonds

Index(['ClPr', 'DirPrIdx', 'NetPrIdx', 'PayIR', 'MktValue', 'SetVol',
       'CompanyCode', 'IdxCd', 'IdxNm', 'IdxType', 'RSecuCode', 'SecuMkt',
       'TrdDt', 'Mat', 'ChgPCT', 'DirPrIdxChgPCT', 'NetPrIdxChgPCT', 'Yield',
       'PreClPr', 'AvgBPVal', 'BdIdxType', 'CptType', 'Conv1', 'Dur1',
       'PreDirPrIdx', 'PreNetPrIdx', 'Conv2', 'Dur2', 'YldMktVal', '交易日期TrdDt',
       '债券指数类型BdIdxType', '全价指数DirPrIdx', '全价指数涨跌幅DirPrIdxChgPCT',
       '公司代码CompanyCode', '净价指数NetPrIdx', '净价指数涨跌幅NetPrIdxChgPCT',
       '前日全价指数值PreDirPrIdx', '前日净价指数值PreNetPrIdx', '前日财富指数值前日指数值PreClPr',
       '平均到期收益率Yield', '平均基点价值AvgBPVal', '平均市值法久期Dur1', '平均市值法凸性Conv1',
       '平均市值法到期收益率YldMktVal', '平均待偿期限年Mat', '平均派息率PayIR', '平均现金流法久期Dur2',
       '平均现金流法凸性Conv2', '成份证券类别CptType', '指数代码IdxCd', '指数名称IdxNm',
       '指数总市值百万元MktValue', '指数类别IdxType', '现券结算量百万元SetVol', '证券代码RSecuCode',
       '证券市场SecuMkt', '财富指数ClPr', '财富指数涨跌幅ChgPCT'],
      dtype='object')

Index(['指标名称', '中债国开债到期收益率:1个月', '中债国开债到期收益率:10年', '中债国开债到期收益率:5年',
       '中债中短期票据到期收益率AA+:5年', '中债中短期票据到期收益率A+:5年'],
      dtype='object')

Index(['time', 'tb1month', 'tb10yrs', 'tb5yrs', 'cbaap5yrs', 'cbap5yrs',
       'year', 'month'],
      dtype='object')

In [4]:
indexName = pd.DataFrame(raw_index['IdxCd'].value_counts().index)
indexName.columns = ['id']
index_inf = pd.read_csv('~/Downloads/data/infoindex.csv')
index_inf.columns

Index(['公司代码_CompanyCode', '证券代码_R_SecuCode', '指数类别()_IdxType', '指数代码_IdxCd',
       '指数名称_IdxNm', '指数全称_FIdxNm', '指数拼音简称_ChiSpelling', '证券市场_SecuMkt',
       '成份证券类别_CptType', '发布日期_IssDt', '基日_BPeriod', '基点(点)_BP', '加权方式_WghMtd',
       '成份证券数量_CptSum', '成份证券调整周期_CptAdjPd', '指数简介_IdxRmk', '停用日期_EndDt',
       '发布机构代码_IssInsCd', '发布机构名称_IssInsNm', '编制机构代码_CrtInsCd',
       '编制机构名称_CrtInsNm'],
      dtype='object')

In [5]:
#indexName['name'] = indexName['id'].apply(lambda id: index_inf[index_inf['指数代码_IdxCd'] == id]['指数名称_IdxNm'].iloc[0])
#indexName['name'].value_counts 
def get_index_name(id):
    if id =='Y00016':
        return '短融总指数'
    elif id == 'Y00020':
        return '央票总指数'
    else:
        name = index_inf[index_inf['指数代码_IdxCd'] == id]['指数名称_IdxNm'].iloc[0]
        return name
indexName['name'] = indexName['id'].apply(get_index_name)
indexName['name']
#信用债总指数： average return of all corporate bonds


0         总指数_3-5年
1      银行间国债指数_10年
2          七年期国债指数
3          金融债券总指数
4          中短期债券指数
5     银行间国债指数_7-10
6           长期债券指数
7              总指数
8         商业银行债券指数
9          中期票据总指数
10            综合指数
11        新中期票据总指数
12      货币市场基金可投资债
13          企业债总指数
14    银行间国债指数_3-5年
15    金融债券总指数_1-3年
16    金融债券总指数_3-5年
17     金融债券总指数_10年
18       总指数_7-10年
19    银行间国债指数_1-3年
20        总指数_5-7年
21           国债总指数
22      综合指数_7-10年
23           新综合指数
24         银行间国债指数
25    金融债券总指数_7-10
26       综合指数_1-3年
27           短融总指数
28       综合指数_5-7年
29        银行间债券总指数
30       综合指数_3-5年
31         交易所国债指数
32    银行间国债指数_5-7年
33        总指数_1-3年
34          信用债总指数
35      综合指数_10年以上
36    金融债券总指数_5-7年
37       总指数_10年以上
38       综合指数_1年以下
39     七年期国债指数_1年以
40      中债信用增进公司增信
41           央票总指数
42    七年期国债指数_1-3年
Name: name, dtype: object

In [6]:
display(indexName[indexName['name'] == '总指数_7-10年'])
display(indexName[indexName['name'] == '信用债总指数'])
display(indexName[indexName['name'] == '银行间国债指数_7-10'])

,id,name
18,Y01007,总指数_7-10年


,id,name
34,Y00031,信用债总指数


,id,name
5,Y14007,银行间国债指数_7-10


## Choose index from Raw data
### oneMonTbill:  
    source: wind '中债国开债到期收益率:1个月' （国开债交易频率较国债更高，同时国开债的税率和公司债类似，因此这里选择国开债而非国债）  
    columns: ['time', 'year', 'month', 'tb1month', 'OMmonthlyRate']    
    len: 164 (valid)  
    time span: 2006.1 - 2019.9  
    unit: percentage 

In [7]:
#get one-month Tbill rate 
oneMonTbill = pd.DataFrame(tbill[['time', 'year', 'month', 'tb1month']])
oneMonTbill.columns = ['time', 'year', 'month', 'tb1month']
oneMonTbill.iloc[:10]
oneMonTbill['key'] = oneMonTbill['year'].apply(str) + oneMonTbill['month'].apply(str)
oneMonTbill = oneMonTbill.groupby('key').last()
oneMonTbill = oneMonTbill[oneMonTbill['tb1month'] != 0]
oneMonTbill['OMmonthlyRate'] = oneMonTbill.tb1month / 12
oneMonTbill['key'] = oneMonTbill['year'].apply(str) + oneMonTbill['month'].apply(str)
oneMonTbill = oneMonTbill.set_index(oneMonTbill['key'])

### long term corporate bond
    source: resset '总指数_7-10年' 
    columns: ['time', 'year', 'month', 'cbaap5yrs', 'CBmonthlyRate']  
    len: 67 (valid)
    time span: 2013.2 - 2019.9, 2016
    unit: percentage 

In [8]:
#get long term corporate bond
indexBondReturn = raw_index[raw_index['IdxCd']== 'Y01007']
longTermCorporateBond = indexBondReturn[['TrdDt', 'NetPrIdx']]
longTermCorporateBond['time'] = longTermCorporateBond['TrdDt'].apply(lambda x: pd.to_datetime(x))
longTermCorporateBond.iloc[:10]
longTermCorporateBond['year'] = longTermCorporateBond['time'].apply(lambda x: x.year)
longTermCorporateBond['month'] = longTermCorporateBond['time'].apply(lambda x: x.month)
longTermCorporateBond['key'] = longTermCorporateBond['year'].apply(str) + longTermCorporateBond['month'].apply(str)
longTermCorporateBond = longTermCorporateBond.groupby('key').last()
longTermCorporateBond = longTermCorporateBond.sort_values(['year', 'month'])

longTermCorporateBond['LTCB_monthly_returns'] = ((longTermCorporateBond['NetPrIdx'] / longTermCorporateBond['NetPrIdx'].shift(1)) - 1) * 100
longTermCorporateBond['key'] = longTermCorporateBond['year'].apply(str) + longTermCorporateBond['month'].apply(str)
longTermCorporateBond = longTermCorporateBond.set_index(longTermCorporateBond['key'])
longTermCorporateBond.iloc[:10]

/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,TrdDt,NetPrIdx,time,year,month,LTCB_monthly_returns,key
key,,,,,,,
20131,2013-01-31,114.90,2013-01-31,2013,1,NaN,20131
20132,2013-02-28,115.13,2013-02-28,2013,2,0.200174,20132
20133,2013-03-29,115.32,2013-03-29,2013,3,0.165031,20133
20134,2013-04-28,116.34,2013-04-28,2013,4,0.884495,20134
20135,2013-05-31,116.46,2013-05-31,2013,5,0.103146,20135
20136,2013-06-28,115.65,2013-06-28,2013,6,-0.695518,20136
20137,2013-07-31,113.99,2013-07-31,2013,7,-1.435365,20137
20138,2013-08-30,111.37,2013-08-30,2013,8,-2.298447,20138
20139,2013-09-30,111.21,2013-09-30,2013,9,-0.143665,20139


### long term government bond
    source: resset '银行间国债指数_7-10' (为了和公司债匹配，这里采用了7-10年)
    columns: ['time', 'year', 'month', 'tb5yrs', 'TBmonthlyRate']  
    len: 67 (valid)
    time span: 2013.2 - 2019.9, 2016
    unit: percentage 

In [9]:
indexBondReturn = raw_index[raw_index['IdxCd']== 'Y14007']
longTermGovBond = indexBondReturn[['TrdDt', 'NetPrIdx']]
longTermGovBond['time'] = longTermGovBond['TrdDt'].apply(lambda x: pd.to_datetime(x))
longTermGovBond.iloc[:10]
longTermGovBond['year'] = longTermGovBond['time'].apply(lambda x: x.year)
longTermGovBond['month'] = longTermGovBond['time'].apply(lambda x: x.month)
longTermGovBond['key'] = longTermGovBond['year'].apply(str) + longTermGovBond['month'].apply(str)
longTermGovBond = longTermGovBond.groupby('key').last()
longTermGovBond = longTermGovBond.sort_values(['year', 'month'])

longTermGovBond['LTGB_monthly_returns'] = ((longTermGovBond['NetPrIdx'] / longTermGovBond['NetPrIdx'].shift(1)) - 1) * 100
longTermGovBond['key'] = longTermGovBond['year'].apply(str) + longTermGovBond['month'].apply(str)
longTermGovBond = longTermGovBond.set_index(longTermGovBond['key'])
longTermGovBond.iloc[:10]

/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,TrdDt,NetPrIdx,time,year,month,LTGB_monthly_returns,key
key,,,,,,,
20131,2013-01-31,120.73,2013-01-31,2013,1,NaN,20131
20132,2013-02-28,120.82,2013-02-28,2013,2,0.074547,20132
20133,2013-03-29,121.05,2013-03-29,2013,3,0.190366,20133
20134,2013-04-28,122.07,2013-04-28,2013,4,0.842627,20134
20135,2013-05-31,122.08,2013-05-31,2013,5,0.008192,20135
20136,2013-06-28,121.21,2013-06-28,2013,6,-0.712647,20136
20137,2013-07-31,119.40,2013-07-31,2013,7,-1.493276,20137
20138,2013-08-30,116.73,2013-08-30,2013,8,-2.236181,20138
20139,2013-09-30,116.76,2013-09-30,2013,9,0.025700,20139


### Average Return of Corporate Bonds - from index
    1. source: resset '信用债指数' 
    columns: ['time', 'year', 'month', 'cbaap5yrs', 'CBmonthlyRate']  
    len: 213 (valid)
    time span: 2002.1 - 2019.9, 2016
    unit: percentage 
    
    2. value weighted 

In [10]:
indexBondReturn = raw_index[raw_index['IdxCd']== 'Y00031']
avergCorpBond = indexBondReturn[['TrdDt', 'NetPrIdx']]
avergCorpBond['time'] = avergCorpBond['TrdDt'].apply(lambda x: pd.to_datetime(x))
avergCorpBond.iloc[:10]
avergCorpBond['year'] = avergCorpBond['time'].apply(lambda x: x.year)
avergCorpBond['month'] = avergCorpBond['time'].apply(lambda x: x.month)
avergCorpBond = avergCorpBond.sort_values('time')
avergCorpBond.iloc[:10]
avergCorpBond['key'] = avergCorpBond['year'].apply(str) + avergCorpBond['month'].apply(str)
avergCorpBond = avergCorpBond.groupby('key').last()
avergCorpBond['ACB_monthly_returns'] = ((avergCorpBond['NetPrIdx'] / avergCorpBond['NetPrIdx'].shift(1)) - 1) * 100
avergCorpBond.iloc[:10]
avergCorpBond['key'] = avergCorpBond['year'].apply(str) + avergCorpBond['month'].apply(str)
avergCorpBond = avergCorpBond.set_index(avergCorpBond['key'])

/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/judyyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [11]:
#value weighted
data = pd.read_csv(data_address + 'monthly_cb_value_fin_adjReturnStata190901.csv')
data = data.dropna(subset = ['year', 'month'])
data['value_price'] = data['trade_value'] * data['returns']
index = data.groupby('datetime').mean()[['value_price', 'trade_value']]
index['ACB2'] = 100 * index['value_price'] /index['trade_value'] #和之前的单位保持一致（即百分比）

In [12]:
index['datetime'] = index.index
index['datetime'] = index['datetime'].apply(lambda x: pd.to_datetime(x))
index['year'] = index['datetime'].apply(lambda x: x.year)
index['month'] = index['datetime'].apply(lambda x: x.month)
index['key'] = index['year'].apply(int).apply(str) + index['month'].apply(int).apply(str)
index.iloc[:10]

,value_price,trade_value,ACB2,datetime,year,month,key
datetime,,,,,,,
2013-01-01,0.000000,0.000000e+00,NaN,2013-01-01,2013,1,20131
2013-01-01 00:00:00,NaN,0.000000e+00,NaN,2013-01-01,2013,1,20131
2013-02-01 00:00:00,778616.218028,5.285222e+07,1.473195,2013-02-01,2013,2,20132
2013-03-01 00:00:00,308418.631760,2.820378e+07,1.093536,2013-03-01,2013,3,20133
2013-04-01 00:00:00,360409.027506,2.958988e+07,1.218015,2013-04-01,2013,4,20134
2013-05-01 00:00:00,218814.172970,2.564151e+07,0.853359,2013-05-01,2013,5,20135
2013-06-01 00:00:00,205520.593601,2.154480e+07,0.953922,2013-06-01,2013,6,20136
2013-07-01 00:00:00,210741.375913,2.942591e+07,0.716176,2013-07-01,2013,7,20137
2013-08-01 00:00:00,-79663.013625,4.000577e+07,-0.199129,2013-08-01,2013,8,20138


In [13]:
data.columns

Index(['v1', 'unnamed0', 'unnamed01', 'unnamed011', 'v5', 'unnamed0111',
       'aa_minus', 'aa_minus_2', 'unnamed01111', 'bond_type', 'cb_id', 'close',
       'comcd', 'compcode', 'coupon', 'credit_q', 'datetime', 'default_spread',
       'downside_risk', 'downside_risk_2', 'extreme_2', 'high', 'illiq_1',
       'illiq_1_2', 'illiq_2', 'illiq_2_2', 'issue_price', 'key_1', 'kurt',
       'low', 'matdate', 'on_market_date', 'open', 'res_day', 'returns',
       'returns_n', 'returns_p', 'skew', 'total_size', 'total_sizevol',
       'trade_month', 'trade_num', 'trade_value', 'trade_year', 'vol',
       'yields', 'log_trade_value', 'creditrate', 'status_dsr', 'relative_dsr',
       'status_vol', 'downside_risk_3', 'month_coupon', 'renew', 'mu', 'sigma',
       'mu1', 'sigma1', 'mu2', 'sigma2', 'var1', 'var2', 'var3', 'var4',
       'var5', 'var6', 'var7', 'var8', 'var9', 'tradesum', 'amihud', 'amihud1',
       'auto', 'year', 'month', 'timemonth', 'Year', 'Month', 'MonRFRet',
       'Date'

### combine all the index

In [14]:
index_combine = pd.merge(oneMonTbill, longTermCorporateBond, on='key')
index_combine = pd.merge(index_combine, longTermGovBond, on='key')
index_combine = pd.merge(index_combine, avergCorpBond, on='key')
index_combine = pd.merge(index_combine, index, on='key')

index_combine = index_combine[['key', 'OMmonthlyRate', 'LTCB_monthly_returns', 'LTGB_monthly_returns', 'ACB_monthly_returns', 'ACB2']].dropna()

index_combine.describe()

,OMmonthlyRate,LTCB_monthly_returns,LTGB_monthly_returns,ACB_monthly_returns,ACB2
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,0.264975,-0.021952,-0.018972,-0.041992,0.535830
std,0.067901,1.204825,1.051054,1.424793,0.772797
min,0.167500,-2.949677,-2.450541,-3.125628,-1.392073
25%,0.206250,-0.981182,-0.630002,-0.791147,0.115482
50%,0.252500,0.103146,0.050167,-0.071611,0.606336
75%,0.315000,0.878306,0.655849,0.440365,0.962931
max,0.436667,2.368835,2.149311,4.742158,2.818713


### Calculate beta for each bond information

In [15]:
data['key'] = data['year'].apply(int).apply(str) + data['month'].apply(int).apply(str)

data = data.join(index_combine.set_index('key'), on='key')

data.iloc[:10]

,v1,unnamed0,unnamed01,unnamed011,v5,unnamed0111,aa_minus,aa_minus_2,unnamed01111,bond_type,...,ds9,Re_n,Re,value_price,key,OMmonthlyRate,LTCB_monthly_returns,LTGB_monthly_returns,ACB_monthly_returns,ACB2
0,22682.0,1.0,1.0,0.0,0.0,0.0,7.0805,7.0805,0.0,1.0,...,NaN,NaN,NaN,NaN,20131,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,7.4717,7.4717,0.0,1.0,...,NaN,-0.003240,-0.003246,0.000000,20131,NaN,NaN,NaN,NaN,NaN
2,22832.0,9.0,9.0,0.0,0.0,0.0,6.9350,6.9350,0.0,1.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195
3,387.0,10.0,10.0,0.0,0.0,0.0,6.4813,6.4813,0.0,13.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195
4,321.0,8.0,8.0,0.0,0.0,0.0,6.4813,6.4813,0.0,13.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195
5,22683.0,6.0,6.0,0.0,0.0,0.0,6.9350,6.9350,0.0,1.0,...,NaN,0.008131,0.008127,800213.286400,20132,0.265833,0.200174,0.074547,4.344577,1.473195
6,23494.0,3.0,3.0,0.0,0.0,0.0,6.9350,6.9350,0.0,1.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195
7,731.0,5.0,5.0,0.0,0.0,0.0,6.4813,6.4813,0.0,13.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195
8,23056.0,2.0,2.0,0.0,0.0,0.0,7.3679,7.3679,0.0,1.0,...,NaN,0.004181,0.004177,757019.149655,20132,0.265833,0.200174,0.074547,4.344577,1.473195
9,22607.0,7.0,7.0,0.0,0.0,0.0,6.9350,6.9350,0.0,1.0,...,NaN,NaN,NaN,NaN,20132,0.265833,0.200174,0.074547,4.344577,1.473195


In [16]:
#calculate 4 set of factors
data['ftr13_term'] = data['LTGB_monthly_returns'] - data['rf']*100
data['ftr24_term'] = data['LTGB_monthly_returns'] - data['OMmonthlyRate']

data['ftr_def'] = data['LTCB_monthly_returns'] - data['LTGB_monthly_returns']

data['ftr1_ex'] = data['ACB_monthly_returns'] - data['rf']*100
data['ftr2_ex'] = data['ACB_monthly_returns'] - data['OMmonthlyRate']
data['ftr3_ex'] = data['ACB2'] - data['rf']*100
data['ftr4_ex'] = data['ACB2'] - data['OMmonthlyRate']

data['depd13'] = data['returns']*100 - data['rf']*100
data['depd24'] = data['returns']*100 - data['OMmonthlyRate']

In [ ]:
data.describe()

,v1,unnamed0,unnamed01,unnamed011,v5,unnamed0111,aa_minus,aa_minus_2,unnamed01111,bond_type,...,ACB2,ftr13_term,ftr24_term,ftr_def,ftr1_ex,ftr2_ex,ftr3_ex,ftr4_ex,depd13,depd24
count,164688.000000,164688.000000,164688.000000,164688.0,164688.0,164688.0,164688.000000,164688.000000,164688.0,164688.000000,...,135270.000000,135270.000000,135270.000000,135270.000000,135270.000000,135270.000000,135270.000000,135270.000000,159205.000000,130126.000000
mean,96587.163212,94582.994723,94582.994723,0.0,0.0,0.0,6.413300,6.413300,0.0,7.912453,...,0.318118,-0.396831,-0.314363,-0.023322,-0.450080,-0.367612,-0.001976,0.080492,-0.087869,-0.033725
std,59148.254544,59027.471263,59027.471263,0.0,0.0,0.0,0.777211,0.777211,0.0,7.612295,...,0.707467,0.938906,0.939842,0.450668,1.346654,1.349384,0.726296,0.715316,2.392753,2.326104
min,0.000000,0.000000,0.000000,0.0,0.0,0.0,2.296100,2.296100,0.0,1.000000,...,-1.392073,-2.858841,-2.782208,-1.499635,-3.434958,-3.348358,-1.656373,-1.613739,-18.373222,-18.326889
25%,45354.750000,42726.750000,42726.750000,0.0,0.0,0.0,5.960700,5.960700,0.0,1.000000,...,-0.104159,-0.817451,-0.774817,-0.174826,-1.225353,-1.096300,-0.470919,-0.341659,-0.000814,0.051401
50%,90073.500000,89924.500000,89924.500000,0.0,0.0,0.0,6.459900,6.459900,0.0,3.000000,...,0.406821,-0.250033,-0.137490,0.078728,-0.353968,-0.295449,0.050298,0.156088,0.163200,0.221667
75%,153804.250000,145564.500000,145564.500000,0.0,0.0,0.0,6.922000,6.922000,0.0,13.000000,...,0.751361,0.180843,0.234643,0.248389,0.134913,0.242246,0.495628,0.563861,0.373559,0.416228
max,203557.000000,203526.000000,203526.000000,0.0,0.0,0.0,8.747500,8.747500,0.0,34.000000,...,2.818713,1.712411,1.859311,0.766461,4.366358,4.458825,2.462813,2.517880,3.686952,3.751952


In [ ]:
from pyfinance.ols import PandasRollingOLS


cb_id = pd.DataFrame(data['cb_id'].value_counts().index)
cb_id.columns = ['id']
data.index = data['key']
new_data = pd.DataFrame([])

window_n = 12

#BETA 1
#f = cut[['ftr13_term', 'ftr_def', 'ftr1_excess']]

for id in cb_id['id']:
    print('1: ', id)
    cut = data[data['cb_id'] == id]
    if len(cut) <= 12:
        continue
    else:
        cut = cut.sort_values('datetime')
        f = cut[['ftr13_term', 'ftr_def', 'ftr1_ex']]
        model = PandasRollingOLS(y = cut['depd13'], x=f, window=window_n)
        beta = model.beta
        beta.columns = ['beta1_term', 'beta1_def', 'beta1_excess']
        beta['key'] = beta.index
        cut = cut.join(beta.set_index('key'), on='key')
        cut.index = cut['key']
        new_data = new_data.append(cut)

data2 = new_data.copy()
new_data.to_csv(data_address + 'new_data.csv')
#BETA 2
new_data = pd.DataFrame([])
#f = cut[['ftr24_term', 'ftr_def', 'ftr2_ex']]

for id in cb_id['id']:
    print('2: ', id)
    cut = data2[data2['cb_id'] == id]
    if len(cut) <= 12:
        continue
    else:
        cut = cut.sort_values('datetime')
        f = cut[['ftr24_term', 'ftr_def', 'ftr2_ex']]
        model = PandasRollingOLS(y = cut['depd24'], x=f, window=window_n)
        beta = model.beta
        beta.columns = ['beta2_term', 'beta2_def', 'beta2_excess']
        beta['key'] = beta.index
        cut = cut.join(beta.set_index('key'), on='key')
        cut.index = cut['key']
        new_data = new_data.append(cut)

data2 = new_data.copy()
new_data.to_csv(data_address + 'new_data.csv')
#BETA 3
new_data = pd.DataFrame([])
#f = cut[['ftr13_term', 'ftr_def', 'ftr3_excess']]

for id in cb_id['id']:
    print('3: ', id)
    cut = data2[data2['cb_id'] == id]
    if len(cut) <= 12:
        continue
    else:
        cut = cut.sort_values('datetime')
        f = cut[['ftr13_term', 'ftr_def', 'ftr3_ex']]
        model = PandasRollingOLS(y = cut['depd13'], x=f, window=window_n)
        beta = model.beta
        beta.columns = ['beta3_term', 'beta3_def', 'beta3_excess']
        beta['key'] = beta.index
        cut = cut.join(beta.set_index('key'), on='key')
        cut.index = cut['key']
        new_data = new_data.append(cut)

data2 = new_data.copy()
new_data.to_csv(data_address + 'new_data.csv')
#BETA 3
new_data = pd.DataFrame([])
#f = cut[['ftr24_term', 'ftr_def', 'ftr4_excess']]

for id in cb_id['id']:
    print('4: ', id)
    cut = data2[data2['cb_id'] == id]
    if len(cut) <= 12:
        continue
    else:
        cut = cut.sort_values('datetime')
        f = cut[['ftr24_term', 'ftr_def', 'ftr4_ex']]
        model = PandasRollingOLS(y = cut['depd24'], x=f, window=window_n)
        beta = model.beta
        beta.columns = ['beta4_term', 'beta4_def', 'beta4_excess']
        beta['key'] = beta.index
        cut = cut.join(beta.set_index('key'), on='key')
        cut.index = cut['key']
        new_data = new_data.append(cut)


        
new_data.iloc[:30]


1:  124110.0
1:  124130.0
1:  124139.0
1:  124103.0
1:  124122.0
1:  124183.0
1:  124140.0
1:  124143.0
1:  124145.0
1:  124175.0
1:  124136.0
1:  124173.0
1:  124167.0
1:  124165.0
1:  124161.0
1:  124158.0
1:  124160.0
1:  124193.0
1:  124135.0
1:  124125.0
1:  122221.0
1:  124112.0
1:  124123.0
1:  124159.0
1:  124190.0
1:  124189.0
1:  124191.0
1:  124212.0
1:  124206.0
1:  124213.0
1:  124170.0
1:  124169.0
1:  124215.0
1:  124166.0
1:  124229.0
1:  124163.0
1:  124231.0
1:  124207.0
1:  124181.0
1:  124235.0
1:  124156.0
1:  124131.0
1:  122241.0
1:  124147.0
1:  124141.0
1:  124258.0
1:  124162.0
1:  124201.0
1:  124253.0
1:  124203.0
1:  124204.0
1:  124205.0
1:  124285.0
1:  124251.0
1:  124210.0
1:  122249.0
1:  122250.0
1:  124217.0
1:  124218.0
1:  124220.0
1:  124246.0
1:  124222.0
1:  124225.0
1:  124227.0
1:  124241.0
1:  122245.0
1:  124232.0
1:  124240.0
1:  124259.0
1:  124256.0
1:  124142.0
1:  124146.0
1:  124172.0
1:  124171.0
1:  124177.0
1:  124149.0
1:  124278.0

1:  124661.0
1:  124824.0
1:  124660.0
1:  124822.0
1:  124821.0
1:  124872.0
1:  124523.0
1:  124788.0
1:  122310.0
1:  124709.0
1:  124897.0
1:  124153.0
1:  124883.0
1:  124763.0
1:  124791.0
1:  124516.0
1:  124600.0
1:  124878.0
1:  124850.0
1:  124776.0
1:  124888.0
1:  123019.0
1:  124875.0
1:  124742.0
1:  124889.0
1:  124891.0
1:  124707.0
1:  124740.0
1:  124648.0
1:  124485.0
1:  124437.0
1:  124874.0
1:  124852.0
1:  124884.0
1:  124911.0
1:  124894.0
1:  124886.0
1:  124929.0
1:  124930.0
1:  124918.0
1:  124900.0
1:  124926.0
1:  124917.0
1:  124906.0
1:  124873.0
1:  124904.0
1:  124907.0
1:  124938.0
1:  124877.0
1:  124921.0
1:  124860.0
1:  124915.0
1:  124879.0
1:  124937.0
1:  124881.0
1:  124936.0
1:  124914.0
1:  124935.0
1:  124887.0
1:  124848.0
1:  122323.0
1:  124582.0
1:  124946.0
1:  123020.0
1:  124588.0
1:  122268.0
1:  122270.0
1:  122320.0
1:  124492.0
1:  122316.0
1:  124725.0
1:  124643.0
1:  124521.0
1:  124693.0
1:  124689.0
1:  124789.0
1:  124194.0

1:  122491.0
1:  130649.0
1:  122447.0
1:  130177.0
1:  122493.0
1:  130652.0
1:  130659.0
1:  130653.0
1:  130655.0
1:  122324.0
1:  122321.0
1:  124756.0
1:  122319.0
1:  124481.0
1:  130701.0
1:  123080.0
1:  127270.0
1:  127269.0
1:  130451.0
1:  127264.0
1:  127262.0
1:  122238.0
1:  130447.0
1:  123069.0
1:  122430.0
1:  127255.0
1:  130710.0
1:  130711.0
1:  127246.0
1:  130697.0
1:  127230.0
1:  127228.0
1:  130677.0
1:  122499.0
1:  122279.0
1:  127070.0
1:  130672.0
1:  123226.0
1:  130743.0
1:  130678.0
1:  127212.0
1:  130679.0
1:  127119.0
1:  123211.0
1:  130736.0
1:  130737.0
1:  127202.0
1:  122488.0
1:  130712.0
1:  136020.0
1:  136005.0
1:  122467.0
1:  122379.0
1:  122479.0
1:  122366.0
1:  130727.0
1:  136008.0
1:  136014.0
1:  122394.0
1:  122480.0
1:  122481.0
1:  130635.0
1:  122482.0
1:  122369.0
1:  122487.0
1:  122465.0
1:  136025.0
1:  122342.0
1:  136010.0
1:  123255.0
1:  130631.0
1:  122223.0
1:  122486.0
1:  122462.0
1:  122461.0
1:  122483.0
1:  122468.0

1:  136311.0
1:  127411.0
1:  136300.0
1:  136355.0
1:  136321.0
1:  136353.0
1:  130704.0
1:  130981.0
1:  130883.0
1:  130977.0
1:  136257.0
1:  136420.0
1:  127196.0
1:  130980.0
1:  130806.0
1:  124528.0
1:  130984.0
1:  136327.0
1:  130863.0
1:  136336.0
1:  136337.0
1:  130380.0
1:  130978.0
1:  127344.0
1:  136289.0
1:  124539.0
1:  123383.0
1:  136254.0
1:  130983.0
1:  136329.0
1:  136335.0
1:  127424.0
1:  130976.0
1:  136408.0
1:  136406.0
1:  136405.0
1:  136231.0
1:  130945.0
1:  130944.0
1:  127407.0
1:  140004.0
1:  140005.0
1:  136400.0
1:  136398.0
1:  140006.0
1:  136394.0
1:  130965.0
1:  130781.0
1:  136390.0
1:  136407.0
1:  127410.0
1:  136387.0
1:  124669.0
1:  136417.0
1:  127425.0
1:  127426.0
1:  127427.0
1:  136415.0
1:  136414.0
1:  136430.0
1:  136431.0
1:  136411.0
1:  136432.0
1:  127431.0
1:  127419.0
1:  127418.0
1:  130766.0
1:  127415.0
1:  136389.0
1:  136404.0
1:  136354.0
1:  136385.0
1:  130319.0
1:  136372.0
1:  136370.0
1:  136365.0
1:  136368.0

1:  130794.0
1:  139173.0
1:  140270.0
1:  136663.0
1:  136377.0
1:  136371.0
1:  130101.0
1:  136496.0
1:  125995.0
1:  125706.0
1:  130309.0
1:  127081.0
1:  130296.0
1:  136709.0
1:  136710.0
1:  127453.0
1:  136708.0
1:  123286.0
1:  127462.0
1:  127461.0
1:  136632.0
1:  127458.0
1:  127457.0
1:  136707.0
1:  127454.0
1:  130752.0
1:  136706.0
1:  136702.0
1:  139252.0
1:  136699.0
1:  136733.0
1:  136700.0
1:  125990.0
1:  139245.0
1:  136701.0
1:  127441.0
1:  136712.0
1:  136703.0
1:  139195.0
1:  136704.0
1:  139241.0
1:  136636.0
1:  139196.0
1:  127449.0
1:  136711.0
1:  136714.0
1:  139199.0
1:  135878.0
1:  136419.0
1:  136745.0
1:  139226.0
1:  136728.0
1:  136729.0
1:  139215.0
1:  136730.0
1:  139229.0
1:  127165.0
1:  136731.0
1:  136736.0
1:  136735.0
1:  136612.0
1:  136734.0
1:  136746.0
1:  136747.0
1:  136713.0
1:  136721.0
1:  127422.0
1:  139238.0
1:  136715.0
1:  136628.0
1:  136716.0
1:  136718.0
1:  136722.0
1:  136749.0
1:  139205.0
1:  136723.0
1:  136570.0

...

4:  130634.0
4:  123028.0
4:  136778.0
4:  139111.0
4:  122300.0
4:  139104.0
4:  122297.0
4:  136779.0
4:  136780.0
4:  136719.0
4:  136781.0
4:  136772.0
4:  130826.0
4:  122290.0
4:  123086.0
4:  136787.0
4:  136831.0
4:  130933.0
4:  136817.0
4:  136811.0
4:  136764.0
4:  139273.0
4:  139276.0
4:  136763.0
4:  136812.0
4:  139279.0
4:  136813.0
4:  136814.0
4:  136815.0
4:  127353.0
4:  136816.0
4:  136818.0
4:  136809.0
4:  136760.0
4:  136819.0
4:  136759.0
4:  136820.0
4:  136821.0
4:  139260.0
4:  139259.0
4:  139258.0
4:  139256.0
4:  139254.0
4:  139220.0
4:  139251.0
4:  136810.0
4:  139272.0
4:  136822.0
4:  136799.0
4:  136422.0
4:  136789.0
4:  136790.0
4:  136791.0
4:  125558.0
4:  140199.0
4:  136792.0
4:  136793.0
4:  125570.0
4:  136796.0
4:  136797.0
4:  136798.0
4:  136800.0
4:  136765.0
4:  136801.0
4:  136802.0
4:  136803.0
4:  136804.0
4:  136805.0
4:  136806.0
4:  136786.0
4:  139268.0
4:  136807.0
4:  136808.0
4:  136767.0
4:  136766.0
4:  139250.0
4:  139253.0

4:  123289.0
4:  139200.0
4:  122317.0
4:  130873.0
4:  140187.0
4:  136532.0
4:  123304.0
4:  140188.0
4:  139395.0
4:  140256.0
4:  140328.0
4:  140197.0
4:  140262.0
4:  139350.0
4:  139335.0
4:  135643.0
4:  139317.0
4:  130651.0
4:  140176.0
4:  139282.0
4:  130690.0
4:  123089.0
4:  123413.0
4:  130227.0
4:  127624.0
4:  127527.0
4:  139021.0
4:  139385.0
4:  139288.0
4:  139362.0
4:  125385.0
4:  123370.0
4:  125383.0
4:  125382.0
4:  125379.0
4:  123369.0
4:  125374.0
4:  125370.0
4:  125367.0
4:  140198.0
4:  130533.0
4:  127272.0
4:  130507.0
4:  130506.0
4:  130469.0
4:  130448.0
4:  123344.0
4:  123342.0
4:  130888.0
4:  127607.0
4:  123288.0
4:  127611.0
4:  127578.0
4:  127577.0
4:  127587.0
4:  127590.0
4:  127572.0
4:  127627.0
4:  127652.0
4:  127594.0
4:  125522.0
4:  127596.0
4:  127603.0
4:  125531.0
4:  127585.0
4:  127613.0
4:  139165.0
4:  127616.0
4:  136705.0
4:  139227.0
4:  127622.0
4:  125548.0
4:  127550.0
4:  140007.0
4:  127636.0
4:  125351.0
4:  125346.0

4:  130251.0
4:  130250.0
4:  127775.0
4:  127772.0
4:  125514.0
4:  127396.0
4:  127520.0
4:  127519.0
4:  127508.0
4:  127503.0
4:  127500.0
4:  140208.0
4:  127493.0
4:  127485.0
4:  127446.0
4:  127376.0
4:  127584.0
4:  127321.0
4:  127315.0
4:  127300.0
4:  127271.0
4:  127265.0
4:  127260.0
4:  127258.0
4:  127231.0
4:  127064.0
4:  127526.0
4:  127528.0
4:  127531.0
4:  127535.0
4:  127580.0
4:  127574.0
4:  127573.0
4:  127568.0
4:  127564.0
4:  127562.0
4:  127561.0
4:  127558.0
4:  127555.0
4:  127554.0
4:  127552.0
4:  127551.0
4:  127549.0
4:  127548.0
4:  127547.0
4:  127546.0
4:  127544.0
4:  127541.0
4:  127537.0
4:  127063.0
4:  127054.0
4:  127001.0
4:  125546.0
4:  125544.0
4:  125540.0
4:  125539.0
4:  125538.0
4:  125537.0
4:  125536.0
4:  125535.0
4:  125534.0
4:  125533.0
4:  125530.0
4:  125529.0
4:  125528.0
4:  125527.0
4:  125526.0
4:  125525.0
4:  125523.0
4:  125521.0
4:  125520.0
4:  125519.0
4:  125545.0
4:  125547.0
4:  125998.0
4:  125549.0
4:  125996.0

4:  125153.0
4:  125165.0
4:  125396.0
4:  127852.0
4:  123272.0
4:  125154.0
4:  127843.0
4:  125158.0
4:  125159.0
4:  123242.0
4:  127844.0
4:  125160.0
4:  123316.0
4:  127837.0
4:  127833.0
4:  127832.0
4:  123317.0
4:  127601.0
4:  125364.0
4:  127830.0
4:  123329.0
4:  123362.0
4:  127835.0
4:  127836.0
4:  127838.0
4:  127803.0
4:  135076.0
4:  127841.0
4:  127839.0
4:  127906.0
4:  123335.0
4:  127907.0
4:  127814.0
4:  127854.0
4:  123271.0
4:  127857.0
4:  123353.0
4:  127855.0
4:  123361.0
4:  127815.0
4:  123253.0
4:  123347.0
4:  127834.0
4:  123252.0
4:  127856.0
4:  127851.0
4:  123279.0
4:  123280.0
4:  123404.0
4:  125371.0
4:  123411.0
4:  127847.0
4:  123414.0
4:  127840.0
4:  123297.0
4:  123418.0
4:  123295.0
4:  123294.0
4:  123421.0
4:  139407.0
4:  139408.0
4:  123293.0
4:  127848.0
4:  127849.0
4:  123285.0
4:  127850.0
4:  123281.0
4:  125390.0
4:  125146.0
4:  125393.0
4:  125155.0
4:  125144.0
4:  127858.0
4:  125389.0
4:  125147.0
4:  125148.0
4:  125149.0

,v1,unnamed0,unnamed01,unnamed011,v5,unnamed0111,aa_minus,aa_minus_2,unnamed01111,bond_type,...,beta1_excess,beta2_term,beta2_def,beta2_excess,beta3_term,beta3_def,beta3_excess,beta4_term,beta4_def,beta4_excess
key,,,,,,,,,,,,,,,,,,,,,
20131,22682.0,1.0,1.0,0.0,0.0,0.0,7.0805,7.0805,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20132,22683.0,6.0,6.0,0.0,0.0,0.0,6.9350,6.9350,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20133,22684.0,13.0,13.0,0.0,0.0,0.0,6.9543,6.9543,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20134,22685.0,121.0,121.0,0.0,0.0,0.0,6.8876,6.8876,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20135,22686.0,232.0,232.0,0.0,0.0,0.0,6.7807,6.7807,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20136,22687.0,446.0,446.0,0.0,0.0,0.0,6.7431,6.7431,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20137,22688.0,613.0,613.0,0.0,0.0,0.0,6.8345,6.8345,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20138,22689.0,701.0,701.0,0.0,0.0,0.0,7.0427,7.0427,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20139,22690.0,1100.0,1100.0,0.0,0.0,0.0,7.1440,7.1440,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
new_data = pd.read_csv(data_address + 'new_data.csv')

In [21]:
len(new_data)

145190

In [24]:
new_data.describe()

,key,v1,unnamed0,unnamed01,unnamed011,v5,unnamed0111,aa_minus,aa_minus_2,unnamed01111,...,beta1_excess,beta2_term,beta2_def,beta2_excess,beta3_term,beta3_def,beta3_excess,beta4_term,beta4_def,beta4_excess
count,145190.000000,145190.000000,145190.000000,145190.000000,145190.0,145190.0,145190.0,145190.000000,145190.000000,145190.0,...,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000,76176.000000
mean,66778.906791,93353.959749,95482.422419,95482.422419,0.0,0.0,0.0,6.453637,6.453637,0.0,...,-0.045423,0.125466,0.181896,-0.040188,-0.007024,-0.112601,0.511573,-0.005158,-0.104932,0.505010
std,79282.847047,59324.061587,58995.880980,58995.880980,0.0,0.0,0.0,0.733904,0.733904,0.0,...,1.238152,1.700383,2.000086,1.238200,1.022166,2.222865,2.236674,1.018786,2.248001,2.248912
min,20131.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,2.503100,2.503100,0.0,...,-13.613792,-11.881483,-19.319563,-13.992672,-7.469344,-20.167198,-17.438493,-7.567238,-20.296853,-17.134158
25%,20169.000000,42107.250000,43191.750000,43191.750000,0.0,0.0,0.0,5.985300,5.985300,0.0,...,-0.199130,-0.292821,-0.427865,-0.193587,-0.307363,-0.815664,-0.024523,-0.304581,-0.814136,-0.042519
50%,20182.000000,85479.500000,93499.500000,93499.500000,0.0,0.0,0.0,6.473300,6.473300,0.0,...,-0.003062,0.025647,0.029212,0.001537,-0.007175,-0.069514,0.377848,-0.003430,-0.041003,0.361804
75%,201410.000000,154190.750000,145823.750000,145823.750000,0.0,0.0,0.0,6.927000,6.927000,0.0,...,0.210839,0.471835,0.745417,0.218242,0.239367,0.497551,1.145459,0.239983,0.520547,1.142121
max,201812.000000,203204.000000,203525.000000,203525.000000,0.0,0.0,0.0,8.747500,8.747500,0.0,...,11.223361,15.468002,20.832115,11.219802,11.138141,20.802835,16.453671,11.214549,21.633916,16.147277
